In [1]:
import pandas as pd

train_df = pd.read_csv('A3_task1_data_files/train.csv', sep='\t')
dev_df = pd.read_csv('A3_task1_data_files/dev.csv', sep='\t')

dev_df.rename(columns={'setence1': 'sentence1'}, inplace=True)

# Drop rows with null values
train_df.dropna(inplace=True)
dev_df.dropna(inplace=True)

In [2]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch
from transformers import BertTokenizer
import string

import torch
from sentence_transformers import models, SentenceTransformer, losses
from sentence_transformers.readers import InputExample
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

/home/arnav/miniconda3/envs/DLA2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/arnav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/arnav/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/arnav/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/arnav/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()

    # Punctuation removal
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmatized_text = ' '.join([lemmatizer.lemmatize(token) for token in tokens])

    return lemmatized_text

In [4]:
class Dataset_1C(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        sentence1 = self.dataframe.iloc[idx]['sentence1']
        sentence2 = self.dataframe.iloc[idx]['sentence2']
        
        # Preprocess the sentences
        sentence1 = preprocess_text(sentence1)
        sentence2 = preprocess_text(sentence2)

        score = torch.tensor([self.dataframe.iloc[idx]['score'] / 5.0], dtype=torch.float) 

        return sentence1, sentence2, score

In [5]:
# Make dataset instances
train_dataset = Dataset_1C(train_df)
dev_dataset = Dataset_1C(dev_df)

In [6]:
# Make dataloader instances
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=64, shuffle=False)

In [9]:
from sentence_transformers import SentenceTransformer, util
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F

# Load a pre-trained Sentence Transformer model
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Assuming you have a DataLoader ready: train_dataloader
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

def train(model, train_dataloader, optimizer, epochs=4):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for sentences1, sentences2, scores in train_dataloader:
            sentences1, sentences2, scores = sentences1, sentences2, scores.to(device)
            optimizer.zero_grad()

            # Encode the sentence pairs
            embeddings1 = model.encode(sentences1, convert_to_tensor=True, batch_size=len(sentences1))
            embeddings2 = model.encode(sentences2, convert_to_tensor=True, batch_size=len(sentences2))

            embeddings1.to(device)
            embeddings2.to(device)

            # Calculate cosine similarity
            cosine_scores = util.cos_sim(embeddings1, embeddings2)

            # Compute loss
            loss = F.mse_loss(cosine_scores, scores)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch: {epoch+1}, Loss: {total_loss / len(train_dataloader)}")


In [10]:
# Train the model
train(model, train_dataloader, optimizer, epochs=4)

/tmp/ipykernel_22667/918528577.py:34: UserWarning: Using a target size (torch.Size([64, 1])) that is different to the input size (torch.Size([64, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(cosine_scores, scores)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [7]:
from sentence_transformers import SentenceTransformer, models, losses
from transformers import AutoTokenizer, AutoModel

class SBERTFineTuner(torch.nn.Module):
    def __init__(self, model_name, device):
        super(SBERTFineTuner, self).__init__()
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.pooling = models.Pooling(self.model.config.hidden_size)

    def forward(self, encoded_input):
        model_output = self.model(**encoded_input)
        pooled_output = self.pooling(model_output['last_hidden_state'], encoded_input['attention_mask'])
        return pooled_output['sentence_embedding']


In [9]:
from torch.optim import Adam

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model = SBERTFineTuner(model_name, device)
model.to(device)
optimizer = Adam(model.parameters(), lr=1e-5)

for param in model.parameters():
    param.requires_grad = True

In [10]:
epochs = 5

for epoch in range(epochs):
        model.train()
        total_loss = 0
        for sentences1, sentences2, scores in train_dataloader:
            optimizer.zero_grad()
            encoded_input1 = model.tokenizer(sentences1, padding=True, truncation=True, return_tensors="pt", max_length=128)
            encoded_input2 = model.tokenizer(sentences2, padding=True, truncation=True, return_tensors="pt", max_length=128)
            embeddings1 = model(encoded_input1.to(model.device))
            embeddings2 = model(encoded_input2.to(model.device))
            loss = losses.CosineSimilarityLoss(model)(embeddings1, embeddings2, scores)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_dataloader)}')

TypeError: Pooling.forward() takes 2 positional arguments but 3 were given